# Try Random Forest Analysis
### Objective
- Find most important features that contributes to winning a match
- Compare winning strategies for different types of surface

In [109]:
#### imports ####
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import os
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# configurations
#%matplotlib inline  

In [2]:
#### Main ####

# import csv 
file_path = '../data/processed/finals_yr08-17.csv'
df_finals = pd.read_csv(file_path)  

# create save path for figures 
save_path = '../reports/figures/try_random_forest/'

### Get data for winners and losers in all finals 2008 - 2017
- extract colums for winner and loser stats
<br>rearrange the dataframe into winner info and loser info, new column "match_result" --> win / lose

In [4]:
# convert the original df columns into all numbers, 
# i.e. give number alias when they are string if possible, otherwise leave out the column
features_finals = pd.get_dummies(df_finals)

In [10]:
# check column names 
# list(features_finals)

# find & select useful columns for training
list(features_finals).index('tourney_year')
features_finals = features_finals.iloc[:,1:36]

In [92]:
# check column names again 
features_finals = features_finals.reindex(sorted(features_finals.columns), axis=1)
list(features_finals)

# build winner / loser profiles with relavant columns
features_finals_w = pd.concat([features_finals.iloc[:,0:2], features_finals.iloc[:,17:]], axis = 1)
features_finals_l = pd.concat([features_finals.iloc[:,0:20]], axis = 1)

features_finals_w['result'] = 1
features_finals_l['result'] = 0


## unify column names and combine winner- and loser- profiles 
features_finals_w = features_finals_w.rename(index=str, columns={'w_1stIn':'1stIn',
                                             'w_1stWon':'1stWon',
                                             'w_2ndWon':'2ndWon',
                                             'w_SvGms':'SvGms',
                                             'w_ace':'ace',
                                             'w_bpFaced':'bpFaced',
                                             'w_bpSaved':'bpSaved',
                                             'w_df':'df',
                                             'w_svpt':'svpt',
                                             'winner_age':'age',
                                             'winner_ht':'ht',
                                             'winner_id':'id',
                                             'winner_rank':'rank',
                                             'winner_rank_points':'rank_points',
                                             'winner_seed':'seed'})

features_finals_l = features_finals_l.rename(index=str, columns={'l_1stIn':'1stIn',
                                             'l_1stWon':'1stWon',
                                             'l_2ndWon':'2ndWon',
                                             'l_SvGms':'SvGms',
                                             'l_ace':'ace',
                                             'l_bpFaced':'bpFaced',
                                             'l_bpSaved':'bpSaved',
                                             'l_df':'df',
                                             'l_svpt':'svpt',
                                             'loser_age':'age',
                                             'loser_ht':'ht',
                                             'loser_id':'id',
                                             'loser_rank':'rank',
                                             'loser_rank_points':'rank_points',
                                             'loser_seed':'seed'})

features_finals_w = features_finals_w.reindex(sorted(features_finals_w.columns), axis=1)
features_finals_l = features_finals_l.reindex(sorted(features_finals_l.columns), axis=1)



In [94]:
features_finals_wl = pd.concat([features_finals_w, features_finals_l])
features_finals_wl = features_finals_wl.dropna()

### Build the first random forest model for predicting winners and losers of final matches
Use data from final matches 2008-2017 to train a random forest model that predicts whether a player with certain match statistics will win or lose the game

- target: "result" column: the player won or lost the game
- features: first try all the columns

<br>Note: This is a very rudimentary model, the player stats are all mixed up, but in reality we should pair stats from both players in a match in order to predict the winner. Try the simple model first, if the predictions are bad, try to include stats of the opponent as well

In [95]:
y = features_finals_wl.result   # define the prediction target
X = features_finals_wl.drop(columns=['result'])   # define the features
# X.head(10)

In [104]:
# Define model. 
# Specify a number for random_state to ensure same results each run, the number can be random
finals_model = DecisionTreeRegressor(random_state=1)

# Fit model
finals_model.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=1,
           splitter='best')

In [107]:
# use mean absolute error to validate the model with training data itself
predicted_final_result = melbourne_model.predict(X)
mean_absolute_error(y, predicted_final_result)


0.0

here the error is 0 because 

In [110]:
# split data into training and testing data
# fixing a random state number makes sure that the split is always the same when running the script
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

## use mean absolute error to validate the model with testing data
# Define model
melbourne_model = DecisionTreeRegressor()
# Fit model
melbourne_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

0.337662337662
